[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/cli/singlepoint.ipynb)

# Single Point

`janus-core` contains various machine learnt interatomic potentials (MLIPs), including MACE based models (MACE-MP, MACE-OFF), CHGNet, SevenNet and more, full list on https://github.com/stfc/janus-core.

Other will be added as their utility is proven beyond a specific material.

## Set up environment (optional)

These steps are required for Google Colab, but may work on other systems too:

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall torch torchaudio torchvision numpy -y
# ! uv pip install janus-core[all] data-tutorials torch==2.5.1 --system
# get_ipython().kernel.do_shutdown(restart=True)

Use `data_tutorials` to get the data required for this tutorial:

In [ ]:
from data_tutorials.data import get_data

get_data(
    url="https://raw.githubusercontent.com/stfc/janus-core/main/docs/source/tutorials/data/",
    filename=["NaCl-set.xyz"],
    folder="../data",
)

## Command-line help and options

Once `janus-core` is installed, the `janus` CLI command should be available:

In [1]:
! janus --help

                                                                                
 Usage: janus [OPTIONS] COMMAND [ARGS]...                                       
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --version                     Print janus version and exit.                  │
│ --install-completion          Install completion for the current shell.      │
│ --show-completion             Show completion for the current shell, to copy │
│                               it or customize the installation.              │
│ --help                        Show this message and exit.                    │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ singlepoint   Perform single point calculations and save to file.            │
│ geomopt       Perform geom

Help for individual `janus` commands also be requested, describing all available options: 

In [2]:
! janus singlepoint --help

                                                                                
 Usage: janus singlepoint [OPTIONS]                                             
                                                                                
 Perform single point calculations and save to file.                            
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ *  --struct                          PATH                Path of structure   │
│                                                          to simulate.        │
│                                                          [default: None]     │
│                                                          [required]          │
│    --arch                            [mace|mace_mp|mace  MLIP architecture   │
│                                      _off|m3gnet|chgnet  to use for          │
│                           

## Running single point calculations

First, we need a file to perform calculations on. Here, we build a periodic salt structure, visualise it, and write out to a file.

<div class="alert alert-block alert-info">
<b>Tip:</b> We use the ASE and WEAS Widget libaries here to build the structure and visualise it.
We discuss these tools in more detail in the Python tutorials. 
</div>

In [3]:
from ase.build import bulk
from ase.io import write
from weas_widget import WeasWidget

NaCl = bulk("NaCl", "rocksalt", a=5.63, cubic=True)

write("../data/NaCl.xyz", NaCl)

v=WeasWidget()
v.from_ase(NaCl)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

WeasWidget(children=(BaseWidget(atoms={'species': {'Na': 'Na', 'Cl': 'Cl'}, 'cell': [5.63, 0.0, 0.0, 0.0, 5.63…

Now we can use an MLIP to run single point calculations. By default, this uses the MACE-MP model:

In [4]:
! janus singlepoint --struct ../data/NaCl.xyz --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

Results from CLI calculations are saved in a newly created directory, `janus_results`:

In [5]:
! ls janus_results/

NaCl-results.extxyz          NaCl-singlepoint-summary.yml
NaCl-singlepoint-log.yml


We can now read the output file back into ASE, to see the saved energy, stresses, and forces:

In [6]:
from ase.io import read

results = read("janus_results/NaCl-results.extxyz")

In [7]:
print(f"Energy: {results.info['mace_mp_energy']}")
print()
print(f"Stress: {results.info['mace_mp_stress']}")
print()
print(f"Forces: {results.arrays['mace_mp_forces']}")

Energy: -27.0300815358917

Stress: [-5.81654699e-03 -5.81654699e-03 -5.81654699e-03  5.97331032e-20
  2.42708931e-19 -2.73642450e-20]

Forces: [[ 0.  0. -0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]
 [-0.  0.  0.]
 [-0. -0. -0.]
 [-0.  0. -0.]
 [ 0.  0. -0.]
 [-0.  0. -0.]]


We can check the units corresponding to these quantities, which are also saved in the "info" dictionary:

In [8]:
print(results.info["units"])

{'energy': 'eV', 'forces': 'ev/Ang', 'stress': 'ev/Ang^3'}


## Using configuration files

All options to `janus` commands can be specified through a YAML input file. These are text files of the form:

```bash
key: value
list_key:
  - list_value_1
  - list_value_2
nested_key_1:
  nested_key_2: nested_value
```

which, in Python, would correspond to a dictionary of the form:

```python
{
    "key": value,
    "list_key": [list_value_1, list_value_2],
    "nested_key": {"nested_key_2": nested_value},
}
```

Although you can specify every option, let's first write a minimal configuration file for `janus singlepoint`:

In [9]:
%%writefile singlepoint_config_1.yml

struct: ../data/NaCl.xyz
tracker: False

Writing singlepoint_config_1.yml


In [10]:
! cat singlepoint_config_1.yml


struct: ../data/NaCl.xyz
tracker: False


We can then use this configuration file to re-run the calculation:

In [11]:
! janus singlepoint --config singlepoint_config_1.yml

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

Options in the configuration files can be overwritten when running a command. For example, the following configuration file also defines the `file_prefix`, which specifies any directories and the prefix of the file names to be output:

In [12]:
%%writefile singlepoint_config_2.yml

struct: ../data/NaCl.xyz
file_prefix: examples/NaCl
tracker: False

Writing singlepoint_config_2.yml


We can now use this configuration file, but replace `file_prefix`:

In [13]:
! janus singlepoint --config singlepoint_config_2.yml --file-prefix outputs/salt

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

<div class="alert alert-block alert-info">
<b>Note:</b> In the CLI, multi-word arguments must be separated by "-".
In configuration files, they are preferably separated by "_", although "-" should be converted automatically.
</div>

This creates a new output directory, `outputs`, as well as starting all output files with "salt-"

In [14]:
! ls outputs

salt-results.extxyz          salt-singlepoint-summary.yml
salt-singlepoint-log.yml


## Running multiple calculations

By default, results from `janus singlepoint` are saved in two forms. For energies, this is an "info" dictionary, and a "results" dictionary.

One set of dictionary keys will be labelled with the MLIP model used ("mace_mp_energy", "mace_mp_stress", "mace_mp_forces"), which ensures these are not overwritten when running calculations with multiple models.

The unlabelled set of keys ("energy", "forces", "stress") allow ASE to use the results for further calculations, but will be overwritten if a new calculation is run.

In [15]:
print(results.info.keys())
print(results.arrays.keys())
print(results.calc.results.keys())

print()
print(results.calc.results["energy"])
print(results.get_potential_energy())
print(results.get_total_energy())

dict_keys(['units', 'model_path', 'arch', 'mace_mp_stress', 'mace_mp_energy', 'system_name'])
dict_keys(['numbers', 'positions', 'mace_mp_forces'])
dict_keys(['forces', 'energy', 'free_energy', 'stress'])

-27.0300815358917
-27.0300815358917
-27.0300815358917


For example, if we re-run the calculation using SevenNet, saving the results to a new output file, we see that "mace_mp_energy" is still saved, other unlabelled results are updated:

In [16]:
! janus singlepoint --struct janus_results/NaCl-results.extxyz --arch sevennet --out janus_results/NaCl-updated-results.extxyz --no-tracker

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

In [17]:
updated_results = read("janus_results/NaCl-updated-results.extxyz")

In [18]:
print(updated_results.info.keys())
print(updated_results.arrays.keys())
print(updated_results.calc.results.keys())

print()
print(updated_results.info["mace_mp_energy"])
print(updated_results.info["sevennet_energy"])
print(updated_results.calc.results["energy"])
print(updated_results.get_potential_energy())
print(updated_results.get_total_energy())

dict_keys(['units', 'model_path', 'arch', 'mace_mp_energy', 'mace_mp_stress', 'system_name', 'sevennet_stress', 'sevennet_energy'])
dict_keys(['numbers', 'positions', 'mace_mp_forces', 'sevennet_forces'])
dict_keys(['energies', 'forces', 'free_energy', 'energy', 'stress'])

-27.0300815358917
-27.057369232177734
-27.057369232177734
-27.057369232177734
-27.057369232177734


## Output files

In addition to the resulting structure file, all calculations run through the CLI generate log (`NaCl-singlepoint-log.yml`) and summary (`NaCl-singlepoint-summary.yml`) files.

In this case, the log file captures timestamps for the start and end of the calculation, but it will also capture any Python warnings generated, and carbon tracking information:

In [19]:
! cat janus_results/NaCl-singlepoint-log.yml


- timestamp: 2025-03-31 15:37:17,643
  level: INFO
  message: 
    - "Starting single point calculation"
  trace: single_point
  line: 355


- timestamp: 2025-03-31 15:37:17,849
  level: INFO
  message: 
    - "Single point calculation complete"
  trace: single_point
  line: 371



The summary file contains:

- The main command run
- The CLI options specified
- Basic information about the structure
- Output files generated by the calculation
- Start and end times of the calculation
- Carbon tracking summary, if applicable 

In [20]:
! cat janus_results/NaCl-singlepoint-summary.yml

command: janus singlepoint
config:
  arch: mace_mp
  calc_kwargs: {}
  device: cpu
  file_prefix: null
  log: null
  model_path: null
  out: null
  properties: []
  read_kwargs: {}
  struct: ../data/NaCl.xyz
  summary: null
  tracker: false
  write_kwargs: {}
info:
  struct:
    formula: Cl4Na4
    n_atoms: 8
    struct_path: ../data/NaCl.xyz
output_files:
  log: /Users/elliottkasoar/Documents/PSDI/janus-core/docs/source/tutorials/cli/janus_results/NaCl-singlepoint-log.yml
  results: /Users/elliottkasoar/Documents/PSDI/janus-core/docs/source/tutorials/cli/janus_results/NaCl-results.extxyz
  summary: /Users/elliottkasoar/Documents/PSDI/janus-core/docs/source/tutorials/cli/janus_results/NaCl-singlepoint-summary.yml
start_time: 31/03/2025, 15:37:17
end_time: 31/03/2025, 15:37:17


## Reusing configuration files

We can extract the configuration used to run our first calculation from the summary file, reusing it with a slight modification.

First, we read in the configuration via the summary file:

In [21]:
import yaml

with open("janus_results/NaCl-singlepoint-summary.yml", encoding="utf8") as file:
    summary = yaml.safe_load(file)

config = summary["config"]
print(config)

{'arch': 'mace_mp', 'calc_kwargs': {}, 'device': 'cpu', 'file_prefix': None, 'log': None, 'model_path': None, 'out': None, 'properties': [], 'read_kwargs': {}, 'struct': '../data/NaCl.xyz', 'summary': None, 'tracker': False, 'write_kwargs': {}}


Next, let's change the structure to run the calculations on, ensure we read all images the file (this is actaully the default for single point calculations), and change the calculated property:

In [22]:
config["struct"] = "../data/NaCl-set.xyz"
config["read_kwargs"] = {"index": ":"} # Key word arguments, passed to ase.io.read`
config["properties"] = ["hessian"] # This must be a list, even for a single quantity

Now, we write this file out, and use it to run our calculation:

In [23]:
with open("singlepoint_config_3.yml", "w", encoding="utf8") as file:
    yaml.dump(config, file)

In [24]:
! cat singlepoint_config_3.yml

arch: mace_mp
calc_kwargs: {}
device: cpu
file_prefix: null
log: null
model_path: null
out: null
properties:
- hessian
read_kwargs:
  index: ':'
struct: ../data/NaCl-set.xyz
summary: null
tracker: false
write_kwargs: {}


In [25]:
! janus singlepoint --config singlepoint_config_3.yml

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

We can read in these results, ensuring we read in all structures from the file:

In [26]:
NaCl_set_results = read("janus_results/NaCl-set-results.extxyz", index=":")

In [27]:
print(NaCl_set_results[0].info["mace_mp_hessian"])

[[[ 1.25015779e+01 -1.01554411e-20  1.38595365e-17]
  [-1.29279560e+01  0.00000000e+00 -3.19988073e-22]
  [ 3.29895005e-01  0.00000000e+00  3.19839641e-18]
  [-2.34182749e-01  0.00000000e+00  0.00000000e+00]
  [-3.13032441e-02  0.00000000e+00 -1.60461922e-17]
  [ 1.96636174e-01  0.00000000e+00 -1.19262239e-18]
  [-3.13032441e-02 -3.46944695e-18  0.00000000e+00]
  [ 1.96636174e-01  1.19262239e-18  0.00000000e+00]]

 [[-5.25714643e-21 -7.62354636e-01  5.11595856e-18]
  [ 0.00000000e+00  1.52481636e+00 -6.50917606e-19]
  [ 0.00000000e+00 -6.53117090e-01  4.98732999e-18]
  [ 0.00000000e+00 -4.14234119e-02 -3.46944695e-18]
  [ 2.16840434e-19 -4.88497760e-03  1.08420217e-19]
  [ 0.00000000e+00 -1.07996124e-01  0.00000000e+00]
  [ 1.40946282e-18  2.02868811e-01  0.00000000e+00]
  [ 0.00000000e+00 -1.57908927e-01 -1.29769071e-23]]

 [[-1.37928486e-17 -8.85611216e-21 -7.62354636e-01]
  [-2.09289921e-20  1.95148002e-19  1.52481636e+00]
  [ 0.00000000e+00  1.73472348e-18 -6.53117090e-01]
  [ 8.67

## Setting keyword arguments

All `janus` commands accept keyword arguments (kwargs), which are input in the form of Python dictionaries: `{"key": value}`.

One useful example is passing options that are specific to an MLIP calculator.

For example, MACE has an option to run with D3 dispersion correction, through the `dispersion` option. In the command-line, this can be set using:

In [28]:
! janus singlepoint --config singlepoint_config_1.yml --calc-kwargs "{'dispersion': True}" --out janus_results/NaCl-dispersion-results.extxyz

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

To avoid the complications of nested quotations, setting this in the configuration file is preferable:

In [29]:
%%writefile singlepoint_config_4.yml

struct: ../data/NaCl.xyz
tracker: False
calc_kwargs:
  dispersion: True
out: janus_results/NaCl-dispersion-results.extxyz

Writing singlepoint_config_4.yml


In [30]:
! janus singlepoint --config singlepoint_config_4.yml

/Users/elliottkasoar/Documents/PSDI/janus-core/.venv/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_i

Comparing the results before and after the correction:

In [31]:
dispersion_results = read("janus_results/NaCl-dispersion-results.extxyz")

print(f"Original results: {results.info['mace_mp_energy']}")
print(f"Results with dispersion correction: {dispersion_results.info['mace_mp_energy']}")

Original results: -27.0300815358917
Results with dispersion correction: -28.749976781125884
